In [30]:
import config
from dataset import MyDataset
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import DataLoader
from model import SimpleYOLO
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import numpy as np
from utils import train_model, evaluate_model, calculate_loss

In [3]:
print(config.data_dir)

../data/kagglehub/datasets/andrewmvd/dog-and-cat-detection/versions/1


In [4]:
print(config.device)

cuda


In [5]:
# data directory
annotations_dir = os.path.join(config.data_dir, 'annotations')
images_dir = os.path.join(config.data_dir, 'images')

# get list of image files and create a dummy dataframe to split the data
image_files = [file for file in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, file))]
df = pd.DataFrame({'image_name': image_files})

print(df.head())

          image_name
0     Cats_Test0.png
1     Cats_Test1.png
2    Cats_Test10.png
3   Cats_Test100.png
4  Cats_Test1000.png


In [6]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
print(train_df.shape)
print(val_df.shape)

(2948, 1)
(738, 1)


In [8]:
# transform
transform = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
# datasets
train_dataset = MyDataset(annotations_dir, images_dir, transform=transform)
val_dataset = MyDataset(annotations_dir, images_dir, transform=transform)

In [16]:
train_dataset.image_files = [file for file in train_dataset.image_files if file in train_df['image_name'].values]
val_dataset.image_files = [file for file in val_dataset.image_files if file in val_df['image_name'].values]

In [17]:
print(train_dataset.__len__())
print(val_dataset.__len__())

2947
738


In [18]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, pin_memory=True)

In [19]:
device = config.device
print(device)

cuda
